This tutorial was taken from an H2O tutorial online: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/starting-h2o.html

In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

h2o.init(max_mem_size=4)


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from c:\users\m\anaconda2\envs\python36\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\m\AppData\Local\Temp\tmp8r46t_2b
  JVM stdout: C:\Users\m\AppData\Local\Temp\tmp8r46t_2b\h2o_m_started_from_python.out
  JVM stderr: C:\Users\m\AppData\Local\Temp\tmp8r46t_2b\h2o_m_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,09 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,"14 days, 20 hours and 35 minutes"
H2O cluster name:,H2O_from_python_m_6gh42v
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


This data is from kaggle when googling 'airline data h2o' because the tutorial file was not a valid web page.

In [2]:

flights2 = h2o.import_file("flights.csv")


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
flights2
flights2.shape

(5819079, 31)

In [4]:
flights2.head()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,US,840,N171US,SFO,CLT,20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,DL,806,N3730B,SFO,MSP,25,20,-5,18,38,217,230,206,1589,604,6,602,610,8,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,NK,612,N635NK,LAS,MSP,25,19,-6,11,30,181,170,154,1299,504,5,526,509,-17,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,US,2013,N584UW,LAX,CLT,30,44,14,13,57,273,249,228,2125,745,8,803,753,-10,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,19,-11,17,36,195,193,173,1464,529,3,545,532,-13,0,0,,nan,nan,nan,nan,nan
2015,1,1,4,DL,1173,N826DN,LAS,ATL,30,33,3,12,45,221,203,186,1747,651,5,711,656,-15,0,0,,nan,nan,nan,nan,nan


In [5]:
flights2.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

In [6]:
flights2['YEAR'] = flights2['YEAR'].asfactor()
flights2['MONTH'] = flights2['MONTH'].asfactor()
flights2['DAY_OF_WEEK'] = flights2['DAY_OF_WEEK'].asfactor()
flights2['FLIGHT_NUMBER'] = flights2['FLIGHT_NUMBER'].asfactor()
flights2['CANCELLED'] = flights2['CANCELLED'].asfactor()
#flights2['DEPARTURE_DELAY'] = flights2['DEPARTURE_DELAY'].asfactor()

In [7]:
predictors = ['YEAR', 'ORIGIN_AIRPORT','DESTINATION_AIRPORT','MONTH', 'DAY_OF_WEEK', 
              'FLIGHT_NUMBER','DISTANCE','AIRLINE']
response = 'DEPARTURE_DELAY'


In [8]:
train, valid = flights2.split_frame(ratios=[0.8], seed=1234)

In [9]:
bin_num = [8,16,32,64,128,256,512,1024,2048,4096]
label = ["8","16","32","64","128","256","512","1024","2048","4096"]

The next command shows the attributes available in the H2OGradientBoostingEstimator function used to train the GBM model and test on the validation set with.

In [10]:
dir(H2OGradientBoostingEstimator)

['_ModelBase__generate_partial_plots',
 '_ModelBase__generate_user_splits',
 '_ModelBase__grabValues',
 '_ModelBase__plot_1dpdp',
 '_ModelBase__plot_2dpdp',
 '_ModelBase__predFor3D',
 '_ModelBase__setAxs1D',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_additional_used_columns',
 '_bc',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_get_metrics',
 '_keyify_if_h2oframe',
 '_metrics_class',
 '_plot',
 '_print_model_scoring_history',
 '_requires_training_frame',
 '_resolve_model',
 '_train',
 '_verify_training_frame_params',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'aucpr',
 'balance_classes',
 'biases',
 'build_tree_one_node',
 'c

The time() give the UTC amount of seconds that have elapsed in floating point values.

In [11]:
import time
start = time.time()
for key, num in enumerate(bin_num):
    flights2_gbm = H2OGradientBoostingEstimator(nbins_cats=num, seed=1234)
    flights2_gbm.train(x=predictors, y=response, training_frame=train, validation_frame=valid)
end = time.time()
predictionTime = (end-start)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


This is an alternate way of reading in the file for python 3.6

In [12]:
flights2_gbm

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1580859444109_10


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,110678.0,5.0,5.0,5.0,25.0,32.0,31.86




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1329.2825288940203
RMSE: 36.459327049385045
MAE: 18.118516320527252
RMSLE: NaN
Mean Residual Deviance: 1329.2825288940203

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 1353.0947850896957
RMSE: 36.784436723833295
MAE: 18.204540718600583
RMSLE: NaN
Mean Residual Deviance: 1353.0947850896957

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2020-02-04 16:22:55,0.016 sec,0.0,37.031308,18.764566,1371.317777,37.278862,18.815101,1389.713527
1,,2020-02-04 16:23:10,15.642 sec,3.0,36.896040,18.616960,1361.317750,37.148286,18.670488,1379.995136
2,,2020-02-04 16:23:30,35.627 sec,7.0,36.782714,18.487281,1352.968056,37.043504,18.545082,1372.221222
3,,2020-02-04 16:23:55,1 min 0.051 sec,12.0,36.701522,18.389381,1347.001735,36.970735,18.451143,1366.835236
4,,2020-02-04 16:24:20,1 min 25.398 sec,17.0,36.646788,18.324858,1342.987095,36.923406,18.389741,1363.337878
5,,2020-02-04 16:24:47,1 min 52.681 sec,22.0,36.605118,18.275130,1339.934657,36.889600,18.343364,1360.842615
6,,2020-02-04 16:25:13,2 min 18.714 sec,27.0,36.573955,18.239270,1337.654157,36.865003,18.310246,1359.028479
7,,2020-02-04 16:25:48,2 min 53.146 sec,34.0,36.529584,18.190719,1334.410508,36.831864,18.267077,1356.586210
8,,2020-02-04 16:26:19,3 min 24.133 sec,40.0,36.497495,18.156603,1332.067112,36.808323,18.236518,1354.852620
9,,2020-02-04 16:26:49,3 min 54.804 sec,46.0,36.473456,18.131413,1330.312994,36.792856,18.214934,1353.714281



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,ORIGIN_AIRPORT,319118368.0,1.000000,0.320754
1,MONTH,198638896.0,0.622461,0.199657
2,DESTINATION_AIRPORT,194843408.0,0.610568,0.195842
3,AIRLINE,132232376.0,0.414368,0.132910
4,FLIGHT_NUMBER,76783288.0,0.240611,0.077177
5,DAY_OF_WEEK,67829360.0,0.212552,0.068177
6,DISTANCE,5454050.5,0.017091,0.005482


In [26]:
predictionTime

2393.457242488861

In [61]:
print('The minutes to run the above code on 5.8 million observations using H2O GBM with 10 bins: ', predictionTime/60)

The minutes to run the above code on 5.8 million observations using H2O GBM with 10 bins:  39.89095404148102


In [53]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, tree
from sklearn.metrics import classification_report, confusion_matrix

In [81]:

flights = pd.read_csv('flights.csv', encoding='unicode_escape')

In [82]:
type(flights)

pandas.core.frame.DataFrame

In [83]:
flights.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

In [85]:
flights = flights.astype({"YEAR":'category', "MONTH":'category',"DAY_OF_WEEK":'category',"FLIGHT_NUMBER":'category',"DISTANCE":'category'})

In [86]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
YEAR                   category
MONTH                  category
DAY                    int64
DAY_OF_WEEK            category
AIRLINE                object
FLIGHT_NUMBER          category
TAIL_NUMBER            object
ORIGIN_AIRPORT         object
DESTINATION_AIRPORT    object
SCHEDULED_DEPARTURE    int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE               category
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL      int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED               int64
CANCELLED              int64
CANCELLATION_REASON    object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float6

In [87]:
X = flights[['YEAR','MONTH','DAY_OF_WEEK','FLIGHT_NUMBER','DISTANCE']]
y = flights['DEPARTURE_DELAY']


(4655263, 5)
(1163816, 5)
(4655263,)
(1163816,)


In [95]:
flightsXY = pd.concat([X,y], axis=1)

In [97]:
flightsXY.shape


(5819079, 6)

In [99]:
flightsXY = flightsXY.dropna()
flightsXY.shape

(5732926, 6)

In [102]:
flightsXY

,YEAR,MONTH,DAY_OF_WEEK,FLIGHT_NUMBER,DISTANCE,DEPARTURE_DELAY
0,2015,1,4,98,1448,-11.0
1,2015,1,4,2336,2330,-8.0
2,2015,1,4,840,2296,-2.0
3,2015,1,4,258,2342,-5.0
4,2015,1,4,135,1448,-1.0
...,...,...,...,...,...,...
5819074,2015,12,4,688,2611,-4.0
5819075,2015,12,4,745,1617,-4.0
5819076,2015,12,4,1503,1598,-9.0
5819077,2015,12,4,333,1189,-6.0


In [103]:
X = flightsXY.iloc[:,0:5].values
y = flightsXY.iloc[:,5].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=20)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4586340, 5)
(1146586, 5)
(4586340,)
(1146586,)


In [104]:
X_train

array([[2015, 7, 3, 4098, 946],
       [2015, 5, 3, 2981, 135],
       [2015, 1, 6, 4096, 472],
       ...,
       [2015, 12, 3, 246, 991],
       [2015, 7, 5, 5046, 83],
       [2015, 3, 3, 3380, 351]], dtype=object)

In [105]:
y_train

array([363.,   3.,  -6., ...,  -4.,  -3.,  20.])

In [106]:
knn = KNeighborsClassifier()
knn.fit(X_train,
        y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [107]:
y_pred = knn.predict(X_test)

In [109]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ',accuracy_score(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy:  0.06603516875315066
